# Exploring and cleaning the data from "Chronik flüchtlingsfeindlicher Vorfälle"

In [1]:
import numpy as np
import pandas as pd

## Load data and get first overview

Check the data set.

In [2]:
df = pd.read_csv('data/mut_gegen_rechte_gewalt.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9012 entries, 0 to 9011
Data columns (total 8 columns):
date           9012 non-null object
category       9012 non-null object
city           9012 non-null object
bundesland     9012 non-null object
casualties     611 non-null object
description    9012 non-null object
source         8258 non-null object
source_url     8259 non-null object
dtypes: object(8)
memory usage: 563.3+ KB


In [4]:
df.head()

,date,category,city,bundesland,casualties,description,source,source_url
0,17.05.2019,Tätlicher Übergriff/Körperverletzung,Prenzlau,Brandenburg,1Verletzte_r,Zwei Deutsche haben am Abend zunächst neben ei...,Nordkurier,{'https://www.nordkurier.de/uckermark/junge-ma...
1,04.05.2019,Tätlicher Übergriff/Körperverletzung,Querfurt,Sachsen-Anhalt,NaN,Ein 21-jähriger aus Syrien wurde in der Nacht ...,Mitteldeutsche Zeitung,{'https://www.mz-web.de/saalekreis/staatsschut...
2,01.05.2019,Sonstige Angriffe,Kirchheim,Hessen,NaN,Unbekannte haben in der Nacht Eier gegen die F...,Süddeutsche Zeitung,{'https://www.sueddeutsche.de/muenchen/staatss...
3,31.03.2019,Verdachtsfall,Lübeck,Schleswig-Holstein,1Verletzte_r,Zwei unbekannte Männer haben einen 27-jährigen...,n-tv,{'https://www.n-tv.de/regionales/hamburg-und-s...
4,02.03.2019,Tätlicher Übergriff/Körperverletzung,Leipzig,Sachsen,1Verletzte_r,Eine Gruppe von acht Männern hat am Nachmittag...,Peiner Allgemeine,{'http://www.paz-online.de/Nachrichten/Panoram...


## Data cleaning

In [5]:
df.dtypes

date           object
category       object
city           object
bundesland     object
casualties     object
description    object
source         object
source_url     object
dtype: object

### Date

Change date column to datetime object.

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

### Category

In [7]:
df['category'].value_counts()

Sonstige Angriffe                       6611
Tätlicher Übergriff/Körperverletzung    1482
Kundgebung/Demo                          361
Verdachtsfall                            286
Brandanschlag                            272
Name: category, dtype: int64

As the data has been scraped from a German website, the five category names are in German. We will replace the German categories with the English translation.

In [8]:
mapping_dict = {
    'Tätlicher Übergriff/Körperverletzung': 'Assault and battery',
    'Brandanschlag': 'Arson attack',
    'Kundgebung/Demo': 'Rally/demonstration',
    'Sonstige Angriffe': 'Other attacks',
    'Verdachtsfall': 'Suspected case'
}
df['category'] = df['category'].map(mapping_dict)

In [9]:
df['category'].value_counts()

Other attacks          6611
Assault and battery    1482
Rally/demonstration     361
Suspected case          286
Arson attack            272
Name: category, dtype: int64

### City

In [10]:
df['city'].unique().shape[0]

2505

In [11]:
df['city'].unique()[0:20]

array(['Prenzlau', 'Querfurt', 'Kirchheim', 'Lübeck', 'Leipzig',
       'Ahrensburg', 'Marzahn, Berlin', 'Zittau', 'Mühlhausen',
       'Plattling', 'Stralsund', 'Hebsack, Remshalden',
       'Vaihingen an der Enz', 'Lütten-Klein, Rostock', 'Spremberg',
       'Neubrandenburg', 'Bad Oeynhausen', 'Cottbus', 'Düsseldorf',
       'Magdeburg'], dtype=object)

### Bundesland

In [12]:
df['bundesland'].unique().shape[0]

16

In [13]:
df['bundesland'].unique()

array(['Brandenburg', 'Sachsen-Anhalt', 'Hessen', 'Schleswig-Holstein',
       'Sachsen', 'Berlin', 'Thüringen', 'Bayern',
       'Mecklenburg-Vorpommern', 'Baden-Württemberg',
       'Nordrhein-Westfalen', 'Niedersachsen', 'Rheinland-Pfalz',
       'Saarland', 'Hamburg', 'Bremen'], dtype=object)

### Casualties

In [14]:
df['casualties'].head()

0    1Verletzte_r
1             NaN
2             NaN
3    1Verletzte_r
4    1Verletzte_r
Name: casualties, dtype: object

How many entries actually do contain casualty numbers?

In [15]:
df[df['casualties'].notnull()].shape[0]

611

There are only 611 entries which contain information on the number of casualties. I suppose this can be interpreted than in other cases there were (luckily) no casualties. 

Let's replace the string representation into numerical values by deleting the characters.

In [16]:
df['casualties'] = df['casualties'].str.replace(r' ?(Verletzte)(_r)?', '').astype(float)

In [17]:
df['casualties'].head()

0    1.0
1    NaN
2    NaN
3    1.0
4    1.0
Name: casualties, dtype: float64

### Description

In [18]:
print(*df['description'].head(3), sep='\n\n')

Zwei Deutsche haben am Abend zunächst neben einer Asylunterkunft randaliert. Als Kinder, die in der Asylunterkunft leben, sie aufforderten, dies zu unterlassen, betraten die beiden 21- bzw. 23-Jährigen das Gelände der Unterkunft. Einer von ihnen zückte ein Messer und soll laut Polizei "Stichbewegungen gegen einen tschetschenischen Bewohner ausgeführt haben. Bei der folgenden Rangelei verletzte sich der Tschetschene an der Hand, ein Deutscher erlitt Verletzungen am Bein und musste operiert werden", so die Polizei weiter. Die Kriminalpolizei ermittelt.

Ein 21-jähriger aus Syrien wurde in der Nacht aus einer Gruppe aus fünf oder sechs jungen Deutschen zunächst rassistisch beleidigt und dann auch geschlagen. Als ein 47-jähriger Zeuge dazwischengehen wollte, sollen ihn die Angreifer zurückgestoßen und am Fuß verletzt haben. Der 21-Jährige musste nicht behandelt werden. Die Täter flüchteten, der Staatsschutz ermittelt.

Unbekannte haben in der Nacht Eier gegen die Fassade einer Asylunterkun

### Source

In [19]:
df['source'].head()

0                Nordkurier
1    Mitteldeutsche Zeitung
2       Süddeutsche Zeitung
3                      n-tv
4         Peiner Allgemeine
Name: source, dtype: object

In [20]:
df['source'].unique().shape[0]

596

In [21]:
df['source'].value_counts()[0:10]

Antwort auf eine Kleine Anfrage im Bundestag (Drucksache 18/11298)          1957
Antwort auf eine Kleine Anfrage im Bundestag (Drucksache 18/10213)           762
Bundesregierung                                                              662
Antwort der Bundesregierung (Drucksache 19/144)                              478
Antwort der Bundesregierung (Drucksache 19/146)                              417
Antwort der Bundesregierung auf eine Kleine Anfrage (Drucksache 19/889)      352
Antwort der Bundesregierung auf eine Kleine Anfrage (Drucksache 19/3753)     329
Antwort der Bundesregierung auf eine Kleine Anfrage (Drucksache 19/5516)     324
Antwort der Bundesregierung auf eine Kleine Anfrage (Drucksache 19/2490)     315
Antwort auf eine Kleine Anfrage im Bundestag (Drucksache 19/889)             234
Name: source, dtype: int64

There are 596 different sources, but the most common ones are related to answers from the Bundesregierung. A further inspection reveals that events documented are quite common, too. Therefore we will categorize the variable source into three categories: government, police, others. 

In [22]:
# Initialise column
df['source_category'] = df['source']

# Replace null values by empty string (otherwise boolean indexing will throw an arrow because of boolean indexing)
df.loc[pd.isnull(df['source_category']), 'source_category'] = ''

# Replace respective values
df.loc[df['source_category'].str.contains(r'Anfrage|Bundesregierung'), 'source_category'] = 'government'
df.loc[df['source_category'].str.contains(r'[Pp]olizei'), 'source_category'] = 'police'
df.loc[~df['source_category'].str.contains(r'government|police|^$'), 'source_category'] = 'other'

# Replace null values by empty string
df.loc[df['source_category'] == '', 'source_category'] = ''

In [23]:
df['source_category'].value_counts(dropna = False)

government    6287
other         1683
               754
police         288
Name: source_category, dtype: int64

Out of all sources, the vast amount belong to governmental reports (6,287), and a considerate amount to police reports (288). The category "other" includes mostly media and NGOs. Around 10% of the events do not contain information on the source. 

### Source url

In [24]:
print(*df['source_url'].head(), sep='\n')

{'https://www.nordkurier.de/uckermark/junge-maenner-in-prenzlau-randalieren-1935542005.html'}
{'https://www.mz-web.de/saalekreis/staatsschutz-ermittelt-junger-syrer-rassistisch-beschimpft-und-attackiert-32472432'}
{'https://www.sueddeutsche.de/muenchen/staatsschutz-ermittelt-angriffe-auf-fluechtlinge-und-ein-drohbrief-1.4429977'}
{'https://www.n-tv.de/regionales/hamburg-und-schleswig-holstein/Syrer-mit-Glasflasche-attackiert-Fremdenfeindliches-Motiv-article20957339.html'}
{'http://www.paz-online.de/Nachrichten/Panorama/Auslaenderfeindlicher-Attacke-Acht-betrunkene-Maenner-verpruegeln-Asylbewerber'}


Delete the curly brackets:

In [25]:
df['source_url'] = df['source_url'].str.replace(r"(\{')?('\})?", '')

In [26]:
print(*df['source_url'].head(), sep='\n')

https://www.nordkurier.de/uckermark/junge-maenner-in-prenzlau-randalieren-1935542005.html
https://www.mz-web.de/saalekreis/staatsschutz-ermittelt-junger-syrer-rassistisch-beschimpft-und-attackiert-32472432
https://www.sueddeutsche.de/muenchen/staatsschutz-ermittelt-angriffe-auf-fluechtlinge-und-ein-drohbrief-1.4429977
https://www.n-tv.de/regionales/hamburg-und-schleswig-holstein/Syrer-mit-Glasflasche-attackiert-Fremdenfeindliches-Motiv-article20957339.html
http://www.paz-online.de/Nachrichten/Panorama/Auslaenderfeindlicher-Attacke-Acht-betrunkene-Maenner-verpruegeln-Asylbewerber


### Final checkup

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9012 entries, 0 to 9011
Data columns (total 9 columns):
date               9012 non-null datetime64[ns]
category           9012 non-null object
city               9012 non-null object
bundesland         9012 non-null object
casualties         611 non-null float64
description        9012 non-null object
source             8258 non-null object
source_url         8259 non-null object
source_category    9012 non-null object
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 633.7+ KB


In [28]:
df.dtypes

date               datetime64[ns]
category                   object
city                       object
bundesland                 object
casualties                float64
description                object
source                     object
source_url                 object
source_category            object
dtype: object

In [29]:
df.head()

,date,category,city,bundesland,casualties,description,source,source_url,source_category
0,2019-05-17,Assault and battery,Prenzlau,Brandenburg,1.0,Zwei Deutsche haben am Abend zunächst neben ei...,Nordkurier,https://www.nordkurier.de/uckermark/junge-maen...,other
1,2019-05-04,Assault and battery,Querfurt,Sachsen-Anhalt,NaN,Ein 21-jähriger aus Syrien wurde in der Nacht ...,Mitteldeutsche Zeitung,https://www.mz-web.de/saalekreis/staatsschutz-...,other
2,2019-05-01,Other attacks,Kirchheim,Hessen,NaN,Unbekannte haben in der Nacht Eier gegen die F...,Süddeutsche Zeitung,https://www.sueddeutsche.de/muenchen/staatssch...,other
3,2019-03-31,Suspected case,Lübeck,Schleswig-Holstein,1.0,Zwei unbekannte Männer haben einen 27-jährigen...,n-tv,https://www.n-tv.de/regionales/hamburg-und-sch...,other
4,2019-03-02,Assault and battery,Leipzig,Sachsen,1.0,Eine Gruppe von acht Männern hat am Nachmittag...,Peiner Allgemeine,http://www.paz-online.de/Nachrichten/Panorama/...,other


## Add additional data

### Population by Bundesland (for standardization) 
Source: https://www-genesis.destatis.de/genesis/online

In [30]:
pop = pd.read_csv('data/12411-0010.csv', 
                 sep=";", 
                 skiprows=6, 
                 nrows=16,
                 header=None, 
                 encoding="cp1250",
                 names=['bundesland', 'population'])

In [31]:
df.sort_values('bundesland')['bundesland'].unique()

array(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'], dtype=object)

In [32]:
pop['bundesland']

0          Baden-Württemberg
1                     Bayern
2                     Berlin
3                Brandenburg
4                     Bremen
5                    Hamburg
6                     Hessen
7     Mecklenburg-Vorpommern
8              Niedersachsen
9        Nordrhein-Westfalen
10           Rheinland-Pfalz
11                  Saarland
12                   Sachsen
13            Sachsen-Anhalt
14        Schleswig-Holstein
15                 Thüringen
Name: bundesland, dtype: object

In [33]:
df = df.merge(pop, on='bundesland', how='left')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9012 entries, 0 to 9011
Data columns (total 10 columns):
date               9012 non-null datetime64[ns]
category           9012 non-null object
city               9012 non-null object
bundesland         9012 non-null object
casualties         611 non-null float64
description        9012 non-null object
source             8258 non-null object
source_url         8259 non-null object
source_category    9012 non-null object
population         9012 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 774.5+ KB


### Official statistics on numbers of refugees
Source: https://github.com/muc-fluechtlingsrat/bamf-asylgeschaeftsstatistik

The data was obtained with an R script.

In [35]:
statistik = pd.read_csv('data/asylmonatszahlen.csv')

In [36]:
statistik.head()

,date,n
0,2015-01,25042
1,2015-02,26083
2,2015-03,32054
3,2015-04,27178
4,2015-05,25992


In [37]:
statistik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
date    52 non-null object
n       52 non-null int64
dtypes: int64(1), object(1)
memory usage: 912.0+ bytes


In [38]:
## df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

## Save data set

In [39]:
df.to_csv('data/mut_gegen_rechte_gewalt_clean.csv', index=False)